<a href="https://colab.research.google.com/github/dhth/tour-pics-classifier/blob/master/experiments/exp_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiment #3

Follow-up to [experiment-2](https://github.com/dhth/tour-pics-classifier/blob/master/experiments/exp_2.ipynb).

**Aim:** Generate heatmaps to see what areas influence the most in coming up with a prediction.

## Setup

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
DRIVE_BASE_PATH = "/content/gdrive/My\ Drive/Colab\ Notebooks/"

In [0]:
from fastai import *
from fastai.vision import *

In [0]:
import requests
import base64

In [0]:
with open('img_at.txt') as f:
    ACCESS_TOKEN = f.read().strip()

In [0]:
def create_imgur_album(title,description='',privacy='hidden'):
    url = 'https://api.imgur.com/3/album'
    payload = {
        'title': title,
        'description': description,
        'privacy':privacy}
    files = {}
    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}'
    }
    response = requests.request('POST', url, headers = headers, data = payload, files = files, allow_redirects=False)
    print(f"https://imgur.com/a/{response.json()['data']['id']}")

In [0]:
album_hash = "K61TEc1"

In [0]:
def upload_to_imgur(file_name,post_title,album_hash):
    url = 'https://api.imgur.com/3/image'
    fh = open(file_name, 'rb');
    payload = {'image': base64.b64encode(fh.read()),
              'album':album_hash,
              'type':'base64',
              'title':post_title,
              'looping':False
              }
    files = {}
    headers = {
      'Authorization': f'Bearer {ACCESS_TOKEN}'
    }
    response = requests.request('POST', url, headers = headers, data = payload, files = files, allow_redirects=False)
    return response.json()['data']['link']

In [0]:
def save_to_img(save_name):
    plt.savefig(f'{save_name}.png')
    img_link = upload_to_imgur(f'{save_name}.png',
                    f'{save_name}',album_hash)
    print(f'![]({img_link})')

## Loading saved model

In [0]:
!mkdir -p local_only_data/models/

In [0]:
!cp {DRIVE_BASE_PATH}/saved_models/TR/exp-2-3-classes-740-im-stage-2-3-epochs.pth local_only_data/models/

In [0]:
DATA_PATH = 'local_only_data'

In [0]:
classes = ['landscape', 'people-close-up', 'people-landscape']
data = ImageDataBunch.single_from_classes(DATA_PATH, classes, size=224).normalize(imagenet_stats)
learn = cnn_learner(data, models.resnet34, pretrained=False)

In [0]:
learn.load('exp-2-3-classes-740-im-stage-2-3-epochs');

## Non-class discriminative heatmaps

### Easy images

In [0]:
test_data ={
    "landscape": [
        "https://cdn.tourradar.com/s3/content-pages/89/1200x630/aLxKP8.jpg",
        "https://cdn.tourradar.com/s3/review/750x400/130750_b1fa7c36.jpg",
        "https://cdn.tourradar.com/s3/review/750x400/105003_1c235882.jpg"
        ],
                    
    "people-landscape": [
        "https://cdn.tourradar.com/s3/review/750x400/136416_f4803a33.jpg",
        "https://cdn.tourradar.com/s3/review/750x400/114273_9ab28070.jpg",
        "https://cdn.tourradar.com/s3/review/750x400/127867_dbaa633e.jpg"
        ],
    "people-close-up": [
        "https://cdn.tourradar.com/s3/review/750x400/94503_18341db0.jpg",
        "https://cdn.tourradar.com/s3/review/750x400/97696_90e6a228.jpg",
        "https://cdn.tourradar.com/s3/review/750x400/115007_a769a0fe.jpg"
    ]
}

In [0]:
def fetch_test_pics(image_dict, local_dir_name, data_classes):
    !mkdir -p {local_dir_name}

    for class_name in data_classes:
        for i, el in enumerate(image_dict[class_name]):
            print(f'wget -q "{el}" -O {local_dir_name}/{class_name}_{i}.jpg')
            !wget -q "{el}" -O {local_dir_name}/{class_name}_{i}.jpg

    return Path(local_dir_name)

In [14]:
test_path = fetch_test_pics(test_data, 'local_only_test_data', classes)

wget -q "https://cdn.tourradar.com/s3/content-pages/89/1200x630/aLxKP8.jpg" -O local_only_test_data/landscape_0.jpg
wget -q "https://cdn.tourradar.com/s3/review/750x400/130750_b1fa7c36.jpg" -O local_only_test_data/landscape_1.jpg
wget -q "https://cdn.tourradar.com/s3/review/750x400/105003_1c235882.jpg" -O local_only_test_data/landscape_2.jpg
wget -q "https://cdn.tourradar.com/s3/review/750x400/94503_18341db0.jpg" -O local_only_test_data/people-close-up_0.jpg
wget -q "https://cdn.tourradar.com/s3/review/750x400/97696_90e6a228.jpg" -O local_only_test_data/people-close-up_1.jpg
wget -q "https://cdn.tourradar.com/s3/review/750x400/115007_a769a0fe.jpg" -O local_only_test_data/people-close-up_2.jpg
wget -q "https://cdn.tourradar.com/s3/review/750x400/136416_f4803a33.jpg" -O local_only_test_data/people-landscape_0.jpg
wget -q "https://cdn.tourradar.com/s3/review/750x400/114273_9ab28070.jpg" -O local_only_test_data/people-landscape_1.jpg
wget -q "https://cdn.tourradar.com/s3/review/750x400/127

In [15]:
test_path.ls()

[PosixPath('local_only_test_data/people-close-up_2.jpg'),
 PosixPath('local_only_test_data/people-landscape_2.jpg'),
 PosixPath('local_only_test_data/people-close-up_0.jpg'),
 PosixPath('local_only_test_data/people-landscape_0.jpg'),
 PosixPath('local_only_test_data/landscape_0.jpg'),
 PosixPath('local_only_test_data/people-landscape_1.jpg'),
 PosixPath('local_only_test_data/landscape_2.jpg'),
 PosixPath('local_only_test_data/landscape_1.jpg'),
 PosixPath('local_only_test_data/people-close-up_1.jpg')]

In [0]:
from fastai.callbacks.hooks import *

In [0]:
m = learn.model.eval();

In [0]:
IMG_SIZE = 224

In [0]:
def non_class_discriminative_activations(xb):
    with hook_output(m[0]) as hook_a: 
        preds = m(xb)
    return hook_a

In [0]:
def show_non_class_discriminative_heatmap(x):
    xb,_ = data.one_item(x)
    xb_im = Image(data.denorm(xb)[0])
    xb = xb.cuda()
    hook_a = non_class_discriminative_activations(xb)
    acts  = hook_a.stored[0].cpu()
    avg_acts = acts.mean(0)
    
    _,ax = plt.subplots()
    xb_im.show(ax)
    ax.imshow(avg_acts, alpha=0.6, extent=(0,IMG_SIZE,IMG_SIZE,0),
              interpolation='bilinear', cmap='magma');

In [0]:
def plot_non_class_disc_multi(img_paths):
    
    num_cols = 2
    num_rows = len(img_paths)
    mul = 3
    
    fig,ax = plt.subplots(num_rows,num_cols, figsize=(num_cols*mul,num_rows*mul))

    for i in range(num_rows):
        x = open_image(img_paths[i])
        pred_class,pred_idx,outputs = learn.predict(x)
        xb,_ = data.one_item(x)
        xb_im = Image(data.denorm(xb)[0])
        xb = xb.cuda()
        hook_a = non_class_discriminative_activations(xb)
        acts  = hook_a.stored[0].cpu()
        avg_acts = acts.mean(0)
        xb_im.show(ax[i,0])
        xb_im.show(ax[i,1])
        ax[i,1].set_title(f'Predicted: {pred_class.obj}')
        ax[i,1].imshow(avg_acts, alpha=0.6, extent=(0,IMG_SIZE,IMG_SIZE,0),
                      interpolation='bilinear', cmap='magma');
    plt.tight_layout()

In [0]:
save_name = 'non-class-disc-1'

plot_non_class_disc_multi(test_path.ls()[:6])

plt.savefig(f'{save_name}.png')
img_link = upload_to_imgur(f'{save_name}.png',
                    f'{save_name}',album_hash)
print(f'![]({img_link})')

![](https://i.imgur.com/rjhxMaE.png)

### Tricky Images

In [0]:
tricky_images = [
    "https://cdn.tourradar.com/s3/review/750x400/105139_192ef04f.jpg",
    "https://cdn.tourradar.com/s3/review/750x400/66523_1a1ef7b4.jpg",
    "https://cdn.tourradar.com/s3/review/750x400/109793_29f3718d.jpg",
    "https://cdn.tourradar.com/s3/review/750x400/111981_5385e8e8.jpg",
    "https://cdn.tourradar.com/s3/review/750x400/103167_7f17d844.jpg",
    "https://cdn.tourradar.com/s3/tour/750x400/88570_0ce071f8.jpg",
    "https://cdn.tourradar.com/s3/review/750x400/134784_88c6e7db.jpg",
    "https://cdn.tourradar.com/s3/tour/750x400/28314_11ae05a4.jpg",
    "https://cdn.tourradar.com/s3/review/750x400/79694_47d829bf.jpg"
]

In [0]:
def fetch_tricky_pics(image_list, local_dir_name):
    !mkdir -p {local_dir_name}
    for i, el in enumerate(image_list):
        print(f'wget -q "{el}" -O {local_dir_name}/file_{i}.jpg')
        !wget -q "{el}" -O {local_dir_name}/file_{i}.jpg

    return Path(local_dir_name)

In [51]:
test_path_2 = fetch_tricky_pics(tricky_images, 'local_only_test_data_tricky')

wget -q "https://cdn.tourradar.com/s3/review/750x400/105139_192ef04f.jpg" -O local_only_test_data_tricky/file_0.jpg
wget -q "https://cdn.tourradar.com/s3/review/750x400/66523_1a1ef7b4.jpg" -O local_only_test_data_tricky/file_1.jpg
wget -q "https://cdn.tourradar.com/s3/review/750x400/109793_29f3718d.jpg" -O local_only_test_data_tricky/file_2.jpg
wget -q "https://cdn.tourradar.com/s3/review/750x400/111981_5385e8e8.jpg" -O local_only_test_data_tricky/file_3.jpg
wget -q "https://cdn.tourradar.com/s3/review/750x400/103167_7f17d844.jpg" -O local_only_test_data_tricky/file_4.jpg
wget -q "https://cdn.tourradar.com/s3/tour/750x400/88570_0ce071f8.jpg" -O local_only_test_data_tricky/file_5.jpg
wget -q "https://cdn.tourradar.com/s3/review/750x400/134784_88c6e7db.jpg" -O local_only_test_data_tricky/file_6.jpg
wget -q "https://cdn.tourradar.com/s3/tour/750x400/28314_11ae05a4.jpg" -O local_only_test_data_tricky/file_7.jpg
wget -q "https://cdn.tourradar.com/s3/review/750x400/79694_47d829bf.jpg" -O loc

In [0]:
save_name = 'non-class-disc-2'

plot_non_class_disc_multi(test_path_2.ls()[:8])

plt.savefig(f'{save_name}.png')
img_link = upload_to_imgur(f'{save_name}.png',
                    f'{save_name}',album_hash)
print(f'![]({img_link})')

![](https://i.imgur.com/OQloaw4.png)

## Class-discriminative heatmaps

In [58]:
class_dict = {}
for i,el in enumerate(data.classes):
    class_dict[el.lower()] = i
class_dict

{'landscape': 0, 'people-close-up': 1, 'people-landscape': 2}

In [0]:
def class_discriminative_activations(xb,cat):
    with hook_output(m[0]) as hook_a: 
        with hook_output(m[0], grad=True) as hook_g:
            preds = m(xb)
            preds[0,int(cat)].backward()
    return hook_a,hook_g

In [0]:
def show_class_discriminative_heatmap(path,cat_name,relu=True,figsize=(8,4)):
    
    fig,ax = plt.subplots(1,2, figsize=figsize)
    
    x = open_image(path)
    xb,_ = data.one_item(x)
    xb_im = Image(data.denorm(xb)[0])
    xb = xb.cuda()
    hook_a,hook_g = class_discriminative_activations(xb,class_dict[cat_name])
    acts = hook_a.stored[0].cpu()
    grad = hook_g.stored[0][0].cpu()

    grad_chan = grad.mean(1).mean(1)
    mult = (acts*grad_chan[...,None,None]).mean(0)
    
    if relu:
        mult = F.relu(mult)
    
    xb_im.show(ax[0])
    xb_im.show(ax[1])
    ax[1].imshow(mult, alpha=0.6, extent=(0,IMG_SIZE,IMG_SIZE,0),
              interpolation='bilinear', cmap='magma');
    plt.title(f'Area influencing {cat_name}')
    plt.tight_layout()

In [0]:
show_class_discriminative_heatmap(test_path.ls()[1],'landscape')

save_name = 'class-disc-1'

plt.savefig(f'{save_name}.png')
img_link = upload_to_imgur(f'{save_name}.png',
                    f'{save_name}',album_hash)
print(f'![]({img_link})')

![](https://i.imgur.com/VJ6yOGb.png)

In [0]:
show_class_discriminative_heatmap(test_path.ls()[1],'people-landscape')

save_name = 'class-disc-2'

plt.savefig(f'{save_name}.png')
img_link = upload_to_imgur(f'{save_name}.png',
                    f'{save_name}',album_hash)
print(f'![]({img_link})')

![](https://i.imgur.com/GKoLySZ.png)

In [0]:
show_class_discriminative_heatmap(test_path_2.ls()[0],'landscape')

save_name = 'class-disc-3'

plt.savefig(f'{save_name}.png')
img_link = upload_to_imgur(f'{save_name}.png',
                    f'{save_name}',album_hash)
print(f'![]({img_link})')

![](https://i.imgur.com/eV9B3gT.png)

In [0]:
show_class_discriminative_heatmap(test_path_2.ls()[0],'people-landscape')

save_name = 'class-disc-4'

plt.savefig(f'{save_name}.png')
img_link = upload_to_imgur(f'{save_name}.png',
                    f'{save_name}',album_hash)
print(f'![]({img_link})')

![](https://i.imgur.com/OTuIRTL.png)

In [0]:
show_class_discriminative_heatmap(test_path_2.ls()[6],'landscape')

save_name = 'class-disc-5'

plt.savefig(f'{save_name}.png')
img_link = upload_to_imgur(f'{save_name}.png',
                    f'{save_name}',album_hash)
print(f'![]({img_link})')

![](https://i.imgur.com/a4ibCcA.png)

In [0]:
show_class_discriminative_heatmap(test_path_2.ls()[6],'people-landscape')

save_name = 'class-disc-6'

plt.savefig(f'{save_name}.png')
img_link = upload_to_imgur(f'{save_name}.png',
                    f'{save_name}',album_hash)
print(f'![]({img_link})')

![](https://i.imgur.com/WIRTPxS.png)

In [0]:
show_class_discriminative_heatmap(test_path_2.ls()[4],'people-landscape')

save_name = 'class-disc-7'

plt.savefig(f'{save_name}.png')
img_link = upload_to_imgur(f'{save_name}.png',
                    f'{save_name}',album_hash)
print(f'![]({img_link})')

![](https://i.imgur.com/aoOxmml.png)

In [0]:
show_class_discriminative_heatmap(test_path_2.ls()[4],'landscape')

save_name = 'class-disc-8'

plt.savefig(f'{save_name}.png')
img_link = upload_to_imgur(f'{save_name}.png',
                    f'{save_name}',album_hash)
print(f'![]({img_link})')

![](https://i.imgur.com/8hGLCYX.png)